# 🎙️ Real-Time Multilingual Voice Translator
This notebook records speech, detects the language, translates it into English, accepts a reply, and translates it back to the original language.

## 🧠 Import Libraries and Initialize Components

This section imports and initializes the essential libraries and variables used throughout the notebook:

- `os`: Provides functions to interact with the operating system (e.g., file management).
- `sounddevice`: Used to capture audio from the microphone.
- `scipy.io.wavfile.write`: Saves the recorded audio into a `.wav` file.
- `speech_recognition`: Transcribes audio into text using Google's Speech Recognition API.
- `googletrans.Translator`: A translation engine that translates text between languages.
- `gtts`: Google Text-to-Speech library that converts translated text into spoken audio.
- `IPython.display`: Used to display audio and UI widgets within Jupyter Notebook.
- `ipywidgets`: Enables interactive elements like buttons and text boxes.
- `clear_output`: Clears previous output in a cell, useful for updating UI.

**Initialization:**
- `translator`: An instance of `Translator` to handle language translation.
- `detected_lang`: A global variable to store the detected source language code (e.g., `'es'` for Spanish).
- `audio_filename`: File path to save the recorded voice input (`voice_input.wav`).


In [1]:
import os
import sounddevice as sd
from scipy.io.wavfile import write
import speech_recognition as sr
from googletrans import Translator
from gtts import gTTS
from IPython.display import Audio, display
import ipywidgets as widgets
from IPython.display import clear_output

translator = Translator()
detected_lang = ''
audio_filename = "voice_input.wav"


## 🎤 Voice Recording with Interactive Buttons

This section creates an interactive interface to start and stop voice recording using `ipywidgets`.

### 🧩 Components:
- `duration = 15`: Sets the maximum duration of recording to 15 seconds.
- `recording = []`: Initializes a global list to store audio data.
- `start_button`, `stop_button`: Two buttons for starting and stopping the recording respectively. The stop button is initially disabled.

### 🛠️ Functions:
- `start_recording(b)`: Triggered when the start button is clicked. Begins recording audio using `sounddevice.rec`, stores it in the `recording` variable, and enables the stop button.
- `stop_recording(b)`: Triggered when the stop button is clicked. Stops the recording, writes it to `voice_input.wav`, and disables the stop button again.

### 📤 Output Interface:
- `output`: A text output widget that displays status messages like “Recording started...” and “Recording stopped.”
- `display(widgets.HBox([...]))`: Renders the buttons horizontally.
- `display(output)`: Shows the recording status below the buttons.

This setup ensures a user-friendly and interactive voice recording experience directly in the notebook.


In [10]:
duration =15  # Max duration in seconds
recording = []

start_button = widgets.Button(description="🎙️ Start Recording")
stop_button = widgets.Button(description="⏹️ Stop Recording", disabled=True)
output = widgets.Output()

def start_recording(b):
    global recording
    with output:
        clear_output()
        print("Recording started...")
    recording = sd.rec(int(duration * 16000), samplerate=16000, channels=1, dtype='int16')
    sd.wait()
    stop_button.disabled = False

def stop_recording(b):
    with output:
        clear_output()
        print("Recording stopped.")
    write(audio_filename, 16000, recording)
    stop_button.disabled = True

start_button.on_click(start_recording)
stop_button.on_click(stop_recording)

display(widgets.HBox([start_button, stop_button]))
display(output)


Output()

## 📝 Speech Recognition and Language Detection

This block processes the recorded audio, converts it into text, detects the spoken language, and translates it into English.

### 🛠️ Steps:
- `recognizer = sr.Recognizer()`: Initializes the speech recognizer using the `speech_recognition` library.
- `with sr.AudioFile(audio_filename) as source:`: Loads the saved audio file (`voice_input.wav`) for processing.
- `recognizer.record(source)`: Reads the entire audio file into memory.

### 🔍 Processing:
- `recognizer.recognize_google(audio)`: Sends the audio to Google’s Web Speech API and retrieves the transcribed text.
- `translator.translate(original_text, dest='en')`: Translates the transcribed text into English using the `googletrans` API.
- `translated.src`: Automatically detects the original language and stores its language code (e.g., 'es' for Spanish) in `detected_lang`.

### 📤 Output:
- Displays:
  - The original speech converted to text.
  - The detected language.
  - The translated English version of the speech.

### ⚠️ Error Handling:
- If any issue occurs during recognition or translation, it is caught and displayed as an error message.

This step bridges the gap between spoken input and language understanding, setting the foundation for the translation-reply loop.


In [11]:
recognizer = sr.Recognizer()

with sr.AudioFile(audio_filename) as source:
    audio = recognizer.record(source)

try:
    original_text = recognizer.recognize_google(audio)
    print("🗣️ Detected Speech:", original_text)
    translated = translator.translate(original_text, dest='en')
    detected_lang = translated.src
    print("🌍 Detected Language:", detected_lang)
    print("🔤 Translated to English:", translated.text)
except Exception as e:
    print("❌ Error in recognition or translation:", e)


🗣️ Detected Speech: hola como estas hoy
🌍 Detected Language: es
🔤 Translated to English: Hello, how are you today


## 💬 Reply Interface and Reverse Translation

This section allows the user to respond in English, translates the reply back to the original detected language, and plays the translated audio.

### 🧩 Components:
- `widgets.Text`: Creates an input box for users to type their reply in English.
- `widgets.Button`: A “Submit Reply” button that triggers the translation and audio response.

### 🛠️ Function: `on_submit_reply(b)`
- Clears previous outputs for clarity.
- Retrieves the user's English reply from the text box.
- Uses `googletrans` to translate the English text back into the originally detected language (stored in `detected_lang`).
- Prints the reverse-translated text.
- Converts the translated text into speech using `gTTS` (Google Text-to-Speech).
- Saves the audio to `reply.mp3` and plays it inline using `IPython.display.Audio`.

### 📤 Output:
- Shows the user's English input.
- Displays the translated version in the original language.
- Plays the audio response of the translated reply.

This final interaction loop completes the bidirectional communication, enabling multilingual voice interaction in real-time.


In [13]:
reply_input = widgets.Text(
    placeholder='Type your reply in English...',
    description='Reply:',
    disabled=False
)

submit_button = widgets.Button(description="📤 Submit Reply")

def on_submit_reply(b):
    clear_output(wait=True)
    english_input = reply_input.value
    print("📝 Your Reply:", english_input)
    translated_back = translator.translate(english_input, dest=detected_lang)
    print("🔁 Translated Back:", translated_back.text)
    tts = gTTS(translated_back.text, lang=detected_lang)
    tts.save("reply.mp3")
    display(Audio("reply.mp3"))

submit_button.on_click(on_submit_reply)

display(reply_input, submit_button)
